In [1]:
import sys
sys.executable

'/usr/local/opt/python/bin/python3.7'

In [28]:
import numpy as np
from scipy.spatial import KDTree
import matplotlib.pyplot as plt
import pickle

from collections import defaultdict
from copy import deepcopy
from heapq import *

from gPRM import gPRM

In [3]:
# ECDF function
def ecdf(data):
    xs = np.sort(data)
    ys = np.arange(1, len(xs)+1)/float(len(xs))
    return [xs, ys]

In [4]:
# r function: (1/n * log n)^(1/d)
def r(num_points, dimension):
    return np.power(np.log(num_points) / num_points, 1 / dimension) 

In [ ]:
D = 2
n = 100

x_init = np.array([0.1] * D)
x_goal = np.array([0.9] * D)

num_simulations = 10

In [ ]:
G = gPRM(n, D, x_init, x_goal)

In [ ]:
#output_lengths = []
output_errors = {1: [], 2: [], 3: []}
#output_paths = []

for i in range(num_simulations):
    G.sample_points()
    
    for j in range(1,4):
        r_n = j * r(n, D)
        
        G.run_simulation(r_n)
    
        #output_lengths[-1].append(G.get_length())
        output_errors[j].append(G.get_error())
        #output_paths[-1].append(G.get_path())
        
for k in output_errors.keys():
    output_errors[k] = np.array(output_errors[k]) / G.get_true_distance()

In [ ]:
np.array(output_errors) / G.get_true_distance()

In [11]:
data = {}
for i in [100, 1000, 10000]:
    for d in [2, 3, 4]:
        data[(i,d)] = {}

In [34]:
for D in [2, 3, 4]:
    for n in [10000]:
        output_errors = {1: [], 2: [], 3: []}
        
        x_init = np.array([0.1] * D)
        x_goal = np.array([0.9] * D)

        num_simulations = 50
        
        G = gPRM(n, D, x_init, x_goal)
        
        i = 0
        while i < (num_simulations):
            G.sample_points()

            for j in range(1,4):
                r_n = j * r(n, D)

                G.run_simulation(r_n)
                if G.get_length() < -0.5:
                    continue
                
                output_errors[j].append(G.get_error())
                
            i += 1
        
        for k in output_errors.keys():
            output_errors[k] = np.array(output_errors[k]) / G.get_true_distance()
            
        # dict (n, D) --> dict (j * r) --> np arrays (relative errors)
        data[(n, D)] = output_errors

KeyboardInterrupt: 

In [35]:
data

{(100,
  2): {1: array([0.04006469, 0.00522945, 0.03341032, 0.05442337, 0.01363011,
         0.02131694, 0.0687588 , 0.03118421, 0.02603699, 0.01530714,
         0.02889623, 0.04745936, 0.06985485, 0.03288517, 0.02869293,
         0.02082727, 0.02172345, 0.02903798, 0.01003815, 0.00823325,
         0.08864459, 0.02967914, 0.00961134, 0.01408432, 0.03600221,
         0.01261093, 0.02662871, 0.01860246, 0.04408363, 0.05148483,
         0.08468333, 0.02565892, 0.03615614, 0.01538119, 0.04003752,
         0.10461338, 0.03504005, 0.00759347, 0.01987897, 0.04426759,
         0.01575157, 0.00653998, 0.01818986, 0.01291997, 0.06940706,
         0.02225127, 0.03388991, 0.08545876, 0.01362473, 0.04204081]), 2: array([0.00987443, 0.00012533, 0.00183714, 0.00035702, 0.00240251,
         0.00614262, 0.00075862, 0.00959799, 0.00251085, 0.00619153,
         0.00390526, 0.00244998, 0.01429671, 0.00375603, 0.00100323,
         0.00081698, 0.00063936, 0.0013724 , 0.00048041, 0.00274563,
         0.00946

In [36]:
output_file = {2:{}, 3:{}, 4:{}}

for k in data.keys():
    dim = k[1]
    num = k[0]
    
    output_file[dim][num]  = data[k]

In [37]:
output_file

{2: {100: {1: array([0.04006469, 0.00522945, 0.03341032, 0.05442337, 0.01363011,
          0.02131694, 0.0687588 , 0.03118421, 0.02603699, 0.01530714,
          0.02889623, 0.04745936, 0.06985485, 0.03288517, 0.02869293,
          0.02082727, 0.02172345, 0.02903798, 0.01003815, 0.00823325,
          0.08864459, 0.02967914, 0.00961134, 0.01408432, 0.03600221,
          0.01261093, 0.02662871, 0.01860246, 0.04408363, 0.05148483,
          0.08468333, 0.02565892, 0.03615614, 0.01538119, 0.04003752,
          0.10461338, 0.03504005, 0.00759347, 0.01987897, 0.04426759,
          0.01575157, 0.00653998, 0.01818986, 0.01291997, 0.06940706,
          0.02225127, 0.03388991, 0.08545876, 0.01362473, 0.04204081]),
   2: array([0.00987443, 0.00012533, 0.00183714, 0.00035702, 0.00240251,
          0.00614262, 0.00075862, 0.00959799, 0.00251085, 0.00619153,
          0.00390526, 0.00244998, 0.01429671, 0.00375603, 0.00100323,
          0.00081698, 0.00063936, 0.0013724 , 0.00048041, 0.00274563,
    

In [38]:
# pickling

f = open("data-4_19_20.pkl","wb")
pickle.dump(output_file,f)
f.close()

In [31]:
# unpickling

g = open("data-4_19_20.pkl","rb")
loaded_output_file = pickle.load(g)
g.close()

False